# 第2章：RDD 基本操作

## 學習目標
- 深入理解 RDD (Resilient Distributed Dataset) 概念
- 掌握 Transformation 和 Action 操作
- 學會 RDD 的創建和基本操作
- 理解 Spark 的分區機制

In [ ]:
# 導入必要的模組
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

## 創建 SparkContext

In [ ]:
# 創建 SparkContext
conf = SparkConf().setAppName("RDD基本操作").setMaster("local[*]")
sc = SparkContext(conf=conf)

print("🎯 RDD 基礎操作示範")
print("=" * 30)
print(f"Spark 版本: {sc.version}")
print(f"應用程式名稱: {sc.appName}")

## 創建 RDD

In [ ]:
# 1. 從集合創建 RDD
numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
numbers_rdd = sc.parallelize(numbers)

print(f"原始數據: {numbers}")
print(f"RDD 分區數: {numbers_rdd.getNumPartitions()}")
print(f"RDD 內容: {numbers_rdd.collect()}")

In [ ]:
# 2. 指定分區數創建 RDD
numbers_rdd_4_partitions = sc.parallelize(numbers, 4)
print(f"指定4個分區的RDD分區數: {numbers_rdd_4_partitions.getNumPartitions()}")

# 查看每個分區的內容
def show_partition_content(index, iterator):
    yield f"分區 {index}: {list(iterator)}"

partition_content = numbers_rdd_4_partitions.mapPartitionsWithIndex(show_partition_content).collect()
for content in partition_content:
    print(content)

## Transformation 操作（延遲執行）

In [ ]:
print("🔄 Transformation 操作:")

# 過濾偶數
even_rdd = numbers_rdd.filter(lambda x: x % 2 == 0)
print(f"偶數: {even_rdd.collect()}")

# 映射操作 - 平方
squared_rdd = numbers_rdd.map(lambda x: x ** 2)
print(f"平方: {squared_rdd.collect()}")

# 映射操作 - 轉換為字串
string_rdd = numbers_rdd.map(lambda x: f"數字_{x}")
print(f"字串轉換: {string_rdd.collect()}")

In [ ]:
# 扁平化映射
words = ["hello world", "spark is awesome", "big data processing"]
words_rdd = sc.parallelize(words)
flat_words_rdd = words_rdd.flatMap(lambda x: x.split())

print(f"原始句子: {words}")
print(f"分詞結果: {flat_words_rdd.collect()}")

In [ ]:
# 去重操作
duplicate_numbers = [1, 2, 2, 3, 3, 3, 4, 4, 4, 4]
duplicate_rdd = sc.parallelize(duplicate_numbers)
unique_rdd = duplicate_rdd.distinct()

print(f"原始數據: {duplicate_numbers}")
print(f"去重結果: {unique_rdd.collect()}")

## Action 操作（觸發計算）

In [ ]:
print("⚡ Action 操作:")

# 計數
count = numbers_rdd.count()
print(f"元素總數: {count}")

# 求和
total = numbers_rdd.reduce(lambda x, y: x + y)
print(f"總和: {total}")

# 取前 N 個元素
first_three = numbers_rdd.take(3)
print(f"前三個元素: {first_three}")

# 取前 N 個元素（排序後）
top_three = numbers_rdd.top(3)
print(f"最大的三個元素: {top_three}")

In [ ]:
# 統計信息
stats = numbers_rdd.stats()
print(f"統計信息:")
print(f"  平均值: {stats.mean():.2f}")
print(f"  標準差: {stats.stdev():.2f}")
print(f"  最大值: {stats.max()}")
print(f"  最小值: {stats.min()}")

## 鍵值對 RDD 操作

In [ ]:
print("🔑 鍵值對 RDD 操作:")

# 創建鍵值對
pairs_rdd = numbers_rdd.map(lambda x: (x % 3, x))
print(f"鍵值對: {pairs_rdd.collect()}")

# 按鍵分組
grouped_rdd = pairs_rdd.groupByKey()
grouped_result = grouped_rdd.mapValues(list).collect()
print(f"按鍵分組: {grouped_result}")

# 按鍵求和
sum_by_key = pairs_rdd.reduceByKey(lambda x, y: x + y)
print(f"按鍵求和: {sum_by_key.collect()}")

In [ ]:
# 更實際的例子：單詞計數
text = ["spark is great", "spark is fast", "python is great"]
text_rdd = sc.parallelize(text)

word_count = text_rdd.flatMap(lambda line: line.split()) \
                    .map(lambda word: (word, 1)) \
                    .reduceByKey(lambda a, b: a + b)

print("單詞計數結果:")
for word, count in word_count.collect():
    print(f"  {word}: {count}")

## RDD 的持久化

In [ ]:
# 創建計算複雜的 RDD
complex_rdd = numbers_rdd.map(lambda x: x ** 2).filter(lambda x: x > 25)

print("第一次計算（沒有緩存）:")
print(f"結果: {complex_rdd.collect()}")

# 緩存 RDD
complex_rdd.cache()

print("第二次計算（使用緩存）:")
print(f"結果: {complex_rdd.collect()}")
print(f"計數: {complex_rdd.count()}")

## RDD 的 Lineage（血統）

In [ ]:
# 查看 RDD 的血統信息
print("RDD 血統信息:")
print(complex_rdd.toDebugString().decode())

## 練習題

### 練習1：基本 RDD 操作
給定一個數字列表，完成以下任務：
1. 創建 RDD
2. 找出所有奇數
3. 將每個奇數乘以 3
4. 計算結果的總和

In [ ]:
# 給定數據
data = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

# 在這裡寫你的代碼
# 提示：使用 filter(), map(), reduce() 等操作

# 你的代碼在此


### 練習2：文本處理
給定一些文本數據，完成以下任務：
1. 將文本分詞
2. 過濾掉長度小於 3 的單詞
3. 統計每個單詞的出現次數
4. 找出出現次數最多的前 3 個單詞

In [ ]:
# 給定文本數據
texts = [
    "Apache Spark is a unified analytics engine",
    "Spark provides high-level APIs in Java, Scala, Python and R",
    "Spark runs on Hadoop, Apache Mesos, Kubernetes, standalone",
    "Spark is built on the concept of resilient distributed datasets"
]

# 在這裡寫你的代碼
# 提示：使用 flatMap(), filter(), map(), reduceByKey(), takeOrdered() 等操作

# 你的代碼在此


### 練習3：數據分析
給定銷售數據，完成以下任務：
1. 計算每個產品的總銷售額
2. 找出銷售額最高的產品
3. 計算平均銷售額
4. 找出銷售額高於平均值的產品

In [ ]:
# 給定銷售數據 (產品, 銷售額)
sales_data = [
    ("Laptop", 25000),
    ("Phone", 18000),
    ("Tablet", 15000),
    ("Laptop", 27000),
    ("Phone", 19000),
    ("Watch", 8000),
    ("Tablet", 16000),
    ("Watch", 9000)
]

# 在這裡寫你的代碼
# 提示：使用 reduceByKey(), max(), mean() 等操作

# 你的代碼在此


## 清理資源

In [ ]:
# 取消緩存
complex_rdd.unpersist()

# 停止 SparkContext
sc.stop()
print("✅ RDD 操作示範完成")

## 總結

在本章中，我們學習了：

1. **RDD 的基本概念** - 分散式、容錯的數據集
2. **Transformation vs Action** - 延遲執行 vs 立即執行
3. **基本 RDD 操作** - map, filter, reduce, collect 等
4. **鍵值對 RDD** - groupByKey, reduceByKey 等
5. **RDD 持久化** - cache 和 persist
6. **RDD 血統** - 容錯和重新計算機制

### 關鍵要點
- RDD 是 Spark 的核心數據抽象
- Transformation 操作是延遲執行的
- Action 操作會觸發實際的計算
- 合理使用緩存可以提高性能

### 下一步
- 繼續學習 [第3章：DataFrame 和 Dataset API](03_dataframe_operations.ipynb)
- 了解更高級的數據處理抽象
- 探索結構化數據處理